In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 2f10e9d_x
molecule = {'h2o': 'atmpro', 'co2': 0.0004, 'n2o': 3.2e-07}
band = [3]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0]
  ng_refs = [2, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(10, 250), (600, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.9, 1.7), (1.4, 1.4, 1.4, 1.55, 1.6, 1.66)]
  wgt = [(0.7, 0.7), (0.7, 0.5, 0.5, 0.5, 0.5, 0.5)]
o co2 band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)]
  wgt = [(0.7, 0.3, 0.7), (0

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`       
            layer    coolrg                layer coolrg
pressure                                               
0.000312        1  0.200939                    1    0.0
0.000750        2  0.228723                    2    0.0
0.001052        3  0.309656                    3    0.0
0.001476        4  0.398929                    4    0.0
0.002070        5  0.494751                    5    0.0
0.002904        6  0.595432                    6    0.0
0.004074        7  0.698397                    7    0.0
0.005714        8  0.800126                    8    0.0
0.008015        9  0.896308                    9    0.0
0.011243       10  0.981622                   10    0.0
0.015771       11  1.050275                   11    0.0
0.022122       12  1.097021                   12    0.0
0.031031       13  1.118469                   13    0.0
0.043528       14  1.090461                   14    0.0
0.061057       15  0.998875                   15    0.0
0.085645       16  0.882264                   16    0.0
0.120136       17  0.774700                   17    0.0
0.168516       18  0.686618                   18    0.0
0.236378       19  0.625128                   19    0.0
0.331549       20  0.596931                   20    0.0
0.465100       21  0.615525                   21    0.0
0.652400       22  0.559206                   22    0.0
0.915100       23  0.417947                   23    0.0
1.283650       24  0.312792                   24    0.0
1.800600       25  0.245709                   25    0.0
2.525700       26  0.194792                   26    0.0
3.542800       27  0.151088                   27    0.0
4.969550       28  0.120195                   28    0.0
6.970850       29  0.104585                   29    0.0
9.778100       30  0.092568                   30    0.0
13.715850      31  0.078717                   31    0.0
19.239350      32  0.063914                   32    0.0
26.987250      33  0.057151                   33    0.0
37.855300      34  0.058665                   34    0.0
53.100050      35  0.059094                   35    0.0
73.887500      36  0.058473                   36    0.0
97.662500      37  0.057146                   37    0.0
121.437500     38  0.052769                   38    0.0
145.212500     39  0.047673                   39    0.0
168.987500     40  0.043315                   40    0.0
192.762500     41  0.039009                   41    0.0
216.537500     42  0.034126                   42    0.0
240.312500     43  0.028561                   43    0.0
264.087500     44  0.021656                   44    0.0
287.862500     45  0.012629                   45    0.0
311.637500     46  0.010395                   46    0.0
335.412500     47  0.020523                   47    0.0
359.187500     48  0.034017                   48    0.0
382.962500     49  0.049134                   49    0.0
406.737500     50  0.066663                   50    0.0
430.512500     51  0.084602                   51    0.0
454.287500     52  0.102164                   52    0.0
478.062500     53  0.118805                   53    0.0
501.837500     54  0.134313                   54    0.0
525.612500     55  0.148607                   55    0.0
549.387500     56  0.161567                   56    0.0
573.162500     57  0.173121                   57    0.0
596.937500     58  0.183029                   58    0.0
620.712500     59  0.191065                   59    0.0
644.487500     60  0.197034                   60    0.0
668.262500     61  0.200870                   61    0.0
692.037500     62  0.202717                   62    0.0
715.812500     63  0.202846                   63    0.0
739.587500     64  0.201727                   64    0.0
763.362500     65  0.199966                   65    0.0
787.137500     66  0.198150                   66    0.0
810.912500     67  0.196872                   67    0.0
834.687500     68  0.196199                   68    0.0
858.462500     69  0.194324           

# Fluxes by Layer

CRD                          CLIRAD               
                        flug       fldg      fnetg      flug fldg     fnetg
pressure    level                                                          
0.000000    1     -19.704359   0.000000 -19.704359 -23.59114  0.0 -23.59114
0.000624    2     -19.704394   0.000050 -19.704344 -23.59114  0.0 -23.59114
0.000876    3     -19.704408   0.000071 -19.704337 -23.59114  0.0 -23.59114
0.001229    4     -19.704428   0.000103 -19.704325 -23.59114  0.0 -23.59114
0.001723    5     -19.704454   0.000153 -19.704301 -23.59114  0.0 -23.59114
0.002417    6     -19.704489   0.000229 -19.704260 -23.59114  0.0 -23.59114
0.003391    7     -19.704536   0.000345 -19.704192 -23.59114  0.0 -23.59114
0.004757    8     -19.704598   0.000519 -19.704079 -23.59114  0.0 -23.59114
0.006672    9     -19.704678   0.000781 -19.703897 -23.59114  0.0 -23.59114
0.009359    10    -19.704780   0.001168 -19.703612 -23.59114  0.0 -23.59114
0.013128    11    -19.704905   0.001731 -19.703174 -23.59114  0.0 -23.59114
0.018415    12    -19.705055   0.002539 -19.702516 -23.59114  0.0 -23.59114
0.025830    13    -19.705226   0.003674 -19.701552 -23.59114  0.0 -23.59114
0.036232    14    -19.705404   0.005230 -19.700174 -23.59114  0.0 -23.59114
0.050823    15    -19.705570   0.007281 -19.698289 -23.59114  0.0 -23.59114
0.071291    16    -19.705720   0.009853 -19.695868 -23.59114  0.0 -23.59114
0.100000    17    -19.705842   0.012975 -19.692867 -23.59114  0.0 -23.59114
0.140271    18    -19.705889   0.016718 -19.689172 -23.59114  0.0 -23.59114
0.196760    19    -19.705778   0.021201 -19.684577 -23.59114  0.0 -23.59114
0.275997    20    -19.705350   0.026640 -19.678709 -23.59114  0.0 -23.59114
0.387100    21    -19.704261   0.033408 -19.670853 -23.59114  0.0 -23.59114
0.543100    22    -19.701519   0.042040 -19.659479 -23.59114  0.0 -23.59114
0.761700    23    -19.696762   0.051764 -19.644998 -23.59114  0.0 -23.59114
1.068500    24    -19.691158   0.061349 -19.629809 -23.59114  0.0 -23.59114
1.498800    25    -19.685283   0.071418 -19.613865 -23.59114  0.0 -23.59114
2.102400    26    -19.679196   0.082899 -19.596297 -23.59114  0.0 -23.59114
2.949000    27    -19.673094   0.096332 -19.576762 -23.59114  0.0 -23.59114
4.136600    28    -19.667661   0.112154 -19.555507 -23.59114  0.0 -23.59114
5.802500    29    -19.663487   0.131699 -19.531788 -23.59114  0.0 -23.59114
8.139200    30    -19.660313   0.157473 -19.502839 -23.59114  0.0 -23.59114
11.417000   31    -19.658307   0.191410 -19.466897 -23.59114  0.0 -23.59114
16.014700   32    -19.659022   0.234997 -19.424025 -23.59114  0.0 -23.59114
22.464000   33    -19.666156   0.290959 -19.375197 -23.59114  0.0 -23.59114
31.510500   34    -19.682185   0.368233 -19.313952 -23.59114  0.0 -23.59114
44.200100   35    -19.706317   0.480548 -19.225769 -23.59114  0.0 -23.59114
62.000000   36    -19.740216   0.639049 -19.101167 -23.59114  0.0 -23.59114
85.775000   37    -19.784560   0.848073 -18.936488 -23.59114  0.0 -23.59114
109.550000  38    -19.827966   1.052419 -18.775547 -23.59114  0.0 -23.59114
133.325000  39    -19.873644   1.246711 -18.626933 -23.59114  0.0 -23.59114
157.100000  40    -19.925574   1.432904 -18.492670 -23.59114  0.0 -23.59114
180.875000  41    -19.984614   1.613931 -18.370683 -23.59114  0.0 -23.59114
204.650000  42    -20.051830   1.791008 -18.260822 -23.59114  0.0 -23.59114
228.425000  43    -20.128452   1.963739 -18.164713 -23.59114  0.0 -23.59114
252.200000  44    -20.216624   2.132348 -18.084276 -23.59114  0.0 -23.59114
275.975000  45    -20.320791   2.297506 -18.023286 -23.59114  0.0 -23.59114
299.750000  46    -20.450006   2.462286 -17.987719 -23.59114  0.0 -23.59114
323.525000  47    -20.603526   2.645082 -17.958444 -23.59114  0.0 -23.59114
347.300000  48    -20.767431   2.866786 -17.900645 -23.59114  0.0 -23.59114
371.075000  49    -20.933030   3.128187 -17.804843 -23.59114  0.0 -23.59114
394.850000  50    -21.102454   3.435987 -17.666467 -23.59114  0.0 -23.591

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')